<a href="https://colab.research.google.com/github/anani-a-missinou/Brassica_Metabolomics_defensive-chemical-traits/blob/main/Canopus_consensus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Canopus_consensus

Author: Anani Amegan MISSINOU (INRAE-IGEPP). I reused a notebook from David Elser (University of Strasbourg) : https://doi.org/10.1101/2022.09.12.507566

Script to merge the outputs of all the tools into one big table also get consensus substructures for insilico-tool and propagate canopus within networks

In [4]:
!pip install pandas

In [3]:
import pandas as pd
import io
df = pd.read_csv(io.StringIO(uploaded['filename.csv'].decode('utf-8')))

NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula,CalcExactMolWt
from rdkit.Chem import MCS
from rdkit.Chem import MolFromSmarts
from collections import Counter
from rdkit.Chem.Fraggle import FraggleSim
from rdkit.Chem.rdMolDescriptors import CalcNumAtoms
from rdkit.Chem import rdMolDescriptors
from rdkit.SimDivFilters import rdSimDivPickers
from rdkit import DataStructs
from collections import defaultdict
import numpy as np
from matplotlib.pyplot import hist,xlabel
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.rdBase import BlockLogs
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

def convert(molecule): #repair InChI and convert to smiles
    try:
        if not molecule.startswith("InChI="):
            molecule="InChI="+molecule
        else:
            pass
        mol=Chem.inchi.MolFromInchi(molecule)
        return(Chem.MolToSmiles(mol,kekuleSmiles=True))
    except:
        return None

def smilestomf(smiles):
    try:
        m = Chem.MolFromSmiles(smiles)
        u=CalcMolFormula(m)
        return u
    except:
        return None



#if CalcExactMolWt(Chem.MolFromSmiles(i))<1300



def assignPointsToClusters(picks,fps):
    clusters = defaultdict(list)
    for i,idx in enumerate(picks):
        clusters[i].append(idx)
    sims = np.zeros((len(picks),len(fps)))
    for i in range(len(picks)):
        pick = picks[i]
        sims[i,:] = DataStructs.BulkTanimotoSimilarity(fps[pick],fps)
        sims[i,i] = 0
    best = np.argmax(sims,axis=0)
    for i,idx in enumerate(best):
        if i not in picks:
            clusters[idx].append(i)
    return clusters

def mls(df,columns,thresh = 0.65): # this is the function to calculate the consensus substructure, takes a dataframe, and the columns to select as input
 smiles=[]
 for i in columns:
     smiles=smiles+df[i].dropna().to_list()
 if len(smiles)>4:
  z= [Chem.MolFromSmiles(i) for i in smiles]
  scaffolds=[FraggleSim.generate_fraggle_fragmentation(i) for i in z]
  clean=[]
  block = BlockLogs()
  for i in scaffolds:
      newlist = [word for line in i for word in line.split('.')]
      newlist=list(set(newlist))
      newlist = [w.replace('*', '') for w in newlist]
      mfs=[Chem.MolFromSmiles(i) for i in newlist ]
      clean.append(mfs)
  del block
  clean=[j for i in clean for j in i]
  clean=[i for i in clean if not i == None]
  clean=[Chem.MolToSmiles(i) for i in clean]

  most=Counter(clean).most_common()
  most= [i[0] for i in most]
  if len(most)>50:
      most=most[0:50]
  else:
      pass
  mols = [Chem.MolFromSmiles(smi) for smi in most]
  mols = [i for i in mols if CalcNumAtoms(i) >12]

  fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m,2,2048) for m in mols]

  lp = rdSimDivPickers.LeaderPicker()
  picks = lp.LazyBitVectorPick(fps,len(fps),thresh)
  clusters = assignPointsToClusters(picks,fps)
  sort_clusters= sorted([clusters[i] for i in clusters],key=len, reverse=True)

  mls=[]
  for _,i in enumerate(range(len(sort_clusters))):
      y=[mols[x] for x in sort_clusters[i]]
      if len(y)>1:
          mls.append(Chem.MolToSmiles(MolFromSmarts(MCS.FindMCS(y,timeout=120).smarts)))
      else:
         pass
  return mls
 else:
        return []

columns=["smiles_cfm_db_spec2vec","smiles-sir","smiles_cfm_j","smiles_moldis_j","smiles_moldis","smiles_cfm_db_cosine"]

df_quant = pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/alltissues15072021-quant-py-dil.csv",index_col='row ID', sep=",") # mzmine quant table

#df_motif = pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/MS2LDA-all.csv",index_col='row ID', sep=",") # MS2LDA download

#df_frag=pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/MS2LDA-fragments.csv",index_col='Motif', sep=",") # MS2LDA download

df_network=pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/allcyto.csv",index_col='shared name', sep=",") # curated gnps output exported from cytoscape

df_sirius=pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/formula_identifications-extended.tsv",index_col='ID', sep="\t") # Sirius output

df_cmpid= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/compound_identifications-extended.tsv",index_col='ID', sep="\t") # Sirius output

#df_anno=pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/MS2LDA-annotation.csv",index_col='Name', sep=",") # MS2LDA download

df_moldis= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/MOLDISCOVERY-5561c3b7-view_significant_unique-main.tsv",index_col='Scan', sep="\t") # moldiscovery gnps download

#df_moldis_j= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/MOLDISCOVERY-jassbi.tsv",index_col='Scan', sep="\t") # moldiscovery gnps download

#df_cfm_j= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/jassbi-results.tsv",index_col='Feature_id', sep="\t") # CFM Datase search output

#df_cfm_spec2vec= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/cfm-hits-database_spec2vec.txt",index_col='Feature_id', sep="\t") # CFM Datase search output

#df_cfm_cosine= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/results_cfmdb_alltissues1507.tsv",index_col='Feature_id', sep="\t")

#df_em= pd.read_csv("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/FEATURE-BASED-MOLECULAR-NETWORKING-cf822b6c-15072021/hitsemmanuelfilter.tsv",index_col='Feature_id', sep="\t") # Emmanuel Datase search output

df_cano= pd.read_csv("input/canopus_formula_summary.tsv", sep="\t")  #canopus data

#df_molnet= pd.read_csv("/Users/delser/Git/phd/molnet_v1.tsv",index_col='cluster index', sep="\t") # molnetenhancher output

# Nouvelle section

In [5]:
!ls -l input

total 2808
-rw-r--r-- 1 root root 1183864 Jul 13 22:14 canopus_formula_summary.tsv
-rw-r--r-- 1 root root 1013415 Jul 13 21:49 compound_identifications-leaves-extended.csv
-rw-r--r-- 1 root root  669193 Jul 13 21:49 formula_identifications-leaves-extended.csv
